In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [35]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

# if 'id' in train.columns: train_df = train.drop(columns=['id'])

print(train.head())

    id        first    second     third       label
0   16   621.022461 -2.341116  2.219315   45.386553
1   18   591.331994  1.586732  1.459008   62.487210
2   10   540.675613  0.491446  1.167377   39.327242
3  111  1994.352060 -3.207705  1.264370  199.909617
4  101  1986.083550 -5.137871  1.713998  196.248352


In [38]:
features = ['first', 'second', 'third']

model = LinearRegression()
model.fit(train[features], train['label'])

preds = model.predict(test[features])

submission = pd.DataFrame({
    'id': test['id'],
    'label': preds
})

print(submission.head())

submission.to_csv('submission.csv', index=False)

    id       label
0  108  189.170245
1   67  133.865333
2   31   78.655389
3  119  204.099021
4   42  127.976179
